In [1]:
# Library
THIS_NOTEBOOK='elastic_search'
import pandas as pd
# SEE http://pandas.pydata.org/pandas-docs/stable/options.html#list-of-options
pd.set_option('expand_frame_repr',False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width',1000)
pd.set_option('display.colheader_justify','left')
np = pd.np
np.set_printoptions(linewidth=500,threshold=np.nan)
from elasticsearch import Elasticsearch as ES
es=ES([{'host':'192.168.3.50','port':9200}])

In [ ]:
# Kibana via curl
import requests as R

a="""-XPOST '192.168.3.50:13300/_search?pretty' -d '{
   "query" : {
    ...
   },
   "sort" : [
      {"price" : {"order" : "asc", "mode" : "avg"}}
   ]
}'
"""
all_fields="""-XGET 'http://192.168.3.50:13300/_all/_mapping?pretty'"""
a="""-XGET 'http://192.168.3.50:13300/test/'"""
!curl $a

In [13]:
print es.indices.refresh('ngx_syslog')
print es.cat.indices()

{u'_shards': {u'successful': 1, u'failed': 0, u'total': 1}}
green  open ngx_syslog 1 0  9 0 21.3kb 21.3kb 
green  open syslog-ng  1 0 10 0 18.8kb 18.8kb 
yellow open .kibana    1 1  1 0  2.1kb  2.1kb 



In [ ]:
# Get most recent post -- 'syslog-ng':
es.search('syslog-ng',body={
  "query": {
    "match_all": {}
  },
  "size": 20,
  "sort": [
    {
      "_timestamp": {
        "order": "desc"
      }
    }
  ]
})

In [ ]:
# Get most recent post -- 'ngx_syslog2':
es.indices.refresh('ngx_syslog2')
t=es.search('ngx_syslog2',body={
  "query": {
    "match_all": {}
  },
  "size": 100,
  "sort": [
    {
      "ISODATE": {
        "order": "desc"
      }
    }
  ]
})
msgs=map(lambda s: s['_source']['MSG'],t['hits']['hits'])
dates=map(lambda s: s['_source']['ISODATE'],t['hits']['hits'])
df = pd.DataFrame({'date':dates,'msg':msgs})
df

In [ ]:
t['hits']['hits']

In [ ]:
# Cmd Examples
print es.cat.shards('test')
print es.indices.get_aliases('test')
print es.indices.segments('test')
print es.indices.status('test')
print es.search(index='syslog-ng',fields=['id'])
print es.get(index='syslog-ng',id="AUpv2n1-DVG5Y0kVWwdy",doc_type='_all')
print es.count()
print es.indices.refresh('test')
print es.indices.get_mapping('syslog-ng','_all')
print es.indices.get_field_mapping('test')
# Delete mapping
#es.indices.delete_mapping(index=['ngx_syslog2'],doc_type='_all',master_timeout=3600)

In [ ]:
# Delete Index
es.indices.delete(index=[''],master_timeout=3600)

In [ ]:
t=es.indices.get(index=['ngx_syslog'])
t

In [ ]:
t['ngx_syslog']['mappings']['message']['properties']

In [3]:
# Upgrade Index
a="""
'{ 
    "mappings": {
        "message": {
            "properties"    : {
                "ISODATE"   : { "type": "date"    },
                "HOSTFROM"  : { "type": "string"  },
                "SOURCEIP"  : { "type": "string"  },
                "HOST"      : { "type": "string"  },
                "SOURCE"    : { "type": "string"  },
                "SEQNUM"    : { "type": "integer" },
                "LEVEL"     : { "type": "string"  },
                "TAG"       : { "type": "string"  },
                "PROGRAM"   : { "type": "string"  },
                "PID"       : { "type": "integer" },
                "MSGID"     : { "type": "string"  },
                "MSG"       : { "type": "string"  },
                "SDATA"     : { "type": "string"  }
            }
        }
    }
}'
""".replace('\n','')
!curl -XPOST 'http://192.168.3.50:9200/ngx_syslog' -d $a

{"acknowledged":true}